<a href="https://colab.research.google.com/github/Samiimasmoudii/Deep-Learning-Course/blob/main/KerasTunerOptimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   #RandomSearch with Keras Tuner#

In [1]:
!pip install keras_tuner



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

# Charger le dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normaliser les données (échelle entre 0 et 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Encodage one-hot des labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Définition de la fonction pour construire le modèle
def build_model(hp):
    model = tf.keras.Sequential()

    # Convolutional Layer 1
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_1', values=[32, 64, 96]),
        kernel_size=hp.Choice('kernel_size_1', values=[3, 5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(hp.Choice('dropout_1', values=[0.2, 0.3, 0.4])))

    # Convolutional Layer 2
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Choice('filters_2', values=[64, 128]),
        kernel_size=hp.Choice('kernel_size_2', values=[3, 5]),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(hp.Choice('dropout_2', values=[0.2, 0.3, 0.4])))

    # Fully Connected Layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Choice('dense_units', values=[128, 256, 512]),
        activation='relu'
    ))
    model.add(tf.keras.layers.Dropout(hp.Choice('dropout_3', values=[0.3, 0.4, 0.5])))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))  # 10 classes

    # Compiler le modèle
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Définir le tuner Keras
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Nombre d'essais de combinaisons d'hyperparamètres
    executions_per_trial=2,  # Nombre d'exécutions par essai pour la robustesse
    directory='cifar10_tuning',
    project_name='cnn_tuner'
)

# Recherche des hyperparamètres
tuner.search(
    x_train, y_train,
    epochs=10,
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
)

# Afficher le meilleur hyperparamètre trouvé
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Le meilleur nombre de filtres pour la première couche : {best_hps.get('filters_1')}
Le meilleur nombre de filtres pour la deuxième couche : {best_hps.get('filters_2')}
La meilleure taille de kernel pour la première couche : {best_hps.get('kernel_size_1')}
Taux d'apprentissage optimal : {best_hps.get('learning_rate')}
""")

# Entraîner le meilleur modèle
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x_train, y_train,
    epochs=10,
    validation_split=0.2
)

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


Trial 5 Complete [00h 05m 53s]
val_accuracy: 0.7161500155925751

Best val_accuracy So Far: 0.7161500155925751
Total elapsed time: 00h 22m 07s

Le meilleur nombre de filtres pour la première couche : 64
Le meilleur nombre de filtres pour la deuxième couche : 64
La meilleure taille de kernel pour la première couche : 3
Taux d'apprentissage optimal : 0.001

Epoch 1/10
1250/1250 [==============================] - 19s 14ms/step - loss: 1.5835 - accuracy: 0.4209 - val_loss: 1.3057 - val_accuracy: 0.5348
Epoch 2/10
1250/1250 [==============================] - 17s 14ms/step - loss: 1.2843 - accuracy: 0.5404 - val_loss: 1.1198 - val_accuracy: 0.6076
Epoch 3/10
1250/1250 [==============================] - 17s 14ms/step - loss: 1.1721 - accuracy: 0.5856 - val_loss: 1.0342 - val_accuracy: 0.6368
Epoch 4/10
1250/1250 [==============================] - 17s 14ms/step - loss: 1.1000 - accuracy: 0.6125 - val_loss: 0.9802 - val_accuracy: 0.6591
Epoch 5/10
1250/1250 [==============================] - 18s

**Simulation de Grid Search with Tuner**

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from kerastuner.tuners import RandomSearch

# Charger et préparer les données CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalisation des données
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Encodage one-hot des labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Fonction pour construire le modèle
def build_model(hp):
    model = Sequential()
    # Couche convolutionnelle avec recherche d'hyperparamètres
    model.add(Conv2D(
        filters=hp.Choice('num_filters', [32, 64]),   # Choix des nombres de filtres
        kernel_size=hp.Choice('kernel_size', [3, 5]), # Taille du noyau
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout_rate', [0.2, 0.4]))) # Taux de dropout
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(hp.Choice('dropout_rate', [0.2, 0.4])))
    model.add(Dense(10, activation='softmax'))

    # Compilation avec un taux d'apprentissage variable
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [0.001, 0.0001])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Initialisation du tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy', # Objectif à maximiser
    max_trials=16,            # Nombre total de combinaisons (GridSearch simulé)
    executions_per_trial=1,   # Répétition des essais pour plus de stabilité
    directory='grid_search_tuner',
    project_name='cifar10_tuning'
)

# Définir les données pour la recherche
tuner.search(
    x_train, y_train,
    epochs=5,
    validation_split=0.2,
    batch_size=64
)

# Afficher les meilleurs hyperparamètres
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nMeilleurs hyperparamètres :")
print(f"Nombre de filtres : {best_hps.get('num_filters')}")
print(f"Taille du noyau : {best_hps.get('kernel_size')}")
print(f"Taux de dropout : {best_hps.get('dropout_rate')}")
print(f"Taux d'apprentissage : {best_hps.get('learning_rate')}")

# Entraîner le meilleur modèle
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x_train, y_train,
    epochs=10,
    validation_split=0.2,
    batch_size=64
)

# Évaluation finale sur le jeu de test
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=0)
print(f"\nPrécision finale sur les données de test : {test_acc:.4f}")


Trial 16 Complete [00h 00m 29s]
val_accuracy: 0.6025000214576721

Best val_accuracy So Far: 0.6360999941825867
Total elapsed time: 00h 09m 11s

Meilleurs hyperparamètres :
Nombre de filtres : 64
Taille du noyau : 5
Taux de dropout : 0.2
Taux d'apprentissage : 0.001
Epoch 1/10
625/625 [==============================] - 9s 13ms/step - loss: 1.6389 - accuracy: 0.4083 - val_loss: 1.3972 - val_accuracy: 0.5118
Epoch 2/10
625/625 [==============================] - 8s 12ms/step - loss: 1.3679 - accuracy: 0.5095 - val_loss: 1.2855 - val_accuracy: 0.5443
Epoch 3/10
625/625 [==============================] - 8s 12ms/step - loss: 1.2692 - accuracy: 0.5479 - val_loss: 1.1903 - val_accuracy: 0.5872
Epoch 4/10
625/625 [==============================] - 8s 13ms/step - loss: 1.1735 - accuracy: 0.5814 - val_loss: 1.1678 - val_accuracy: 0.5847
Epoch 5/10
625/625 [==============================] - 8s 12ms/step - loss: 1.0976 - accuracy: 0.6101 - val_loss: 1.1030 - val_accuracy: 0.6155
Epoch 6/10
625/625 